In [1]:
import torch
from torch import nn
import sys
sys.path.append('../fairseq-l-system-captioning/')

from lsystem_modules import SlimLstm

vocab = torch.load('../fairseq-l-system-captioning/data/Advanced_47946_10_max_derivations_vocab.pth')

model = SlimLstm.load_from_checkpoint('../fairseq-l-system-captioning/checkpoints/SlimLstm_normal_rand/epoch=493-step=670852.ckpt')

/home/magnusson/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [2]:
test_cell = nn.LSTMCell(256,256)

In [3]:
test_cell.weight_ih = model.decoder.lstm.weight_ih_l0
test_cell.weight_hh = model.decoder.lstm.weight_hh_l0
test_cell.bias_ih   = model.decoder.lstm.bias_ih_l0
test_cell.bias_hh   = model.decoder.lstm.bias_hh_l0

In [4]:
x = torch.rand((256,))
x.requires_grad = True

In [5]:
h, c = test_cell(x)

In [6]:
from lrp.functional.lstm import LstmAll

In [7]:
test_cell.eps = torch.tensor(0.001)
test_cell.bias_factor = torch.tensor(0.0)

a = torch.zeros(256)

_, h_a, c_a = LstmAll.apply(x[None], a[None], a[None], test_cell)
h_a.retain_grad()

In [8]:
h_a.sum().backward()

In [9]:
from lrp.functional._lstm import LSTM_bidi

lstm = LSTM_bidi(model.decoder.lstm)

In [10]:
Rx = lstm.lrp(x.detach()[None].numpy())
Rx

array([[-1.30045021e+01, -4.41491736e+00, -1.62079134e+01,
         1.09311286e+01,  1.39697825e+01, -1.13479349e+01,
         2.69549008e+00, -2.88247136e+00, -6.00779652e+01,
        -4.46851236e+00,  1.08798199e+00,  1.41147409e+00,
        -7.61768763e+00, -4.88110590e-01,  3.67592801e+00,
         3.29660180e+01,  3.32459252e+00,  3.03552597e+01,
        -2.71590209e+00, -4.99673747e+01,  6.01284358e+00,
        -9.30073290e+00, -2.31253070e+00,  6.90225291e-01,
        -3.20740606e+01,  7.98474856e-01,  6.65627355e+00,
        -2.27609774e+01, -1.11142754e+00,  4.29010793e+00,
         1.01322935e+01, -1.40966006e+01,  1.36676604e+01,
         7.85074860e-01, -6.43635438e+00,  4.24199462e+00,
         4.13203349e+00,  6.32310807e+00, -2.33604078e+01,
         1.35409120e+01, -1.34614729e+01, -1.87217148e+01,
         4.20098466e+01,  5.46570055e+00, -4.47505550e+00,
         1.22349892e+01,  8.23159562e+00,  1.56589112e+01,
        -2.39402529e+00, -4.71774574e+00, -1.32574893e+0

In [16]:
torch.testing.assert_close(h_a.float(), torch.tensor(lstm.s)[None].float())

In [17]:
R_x = x.grad[None]
torch.testing.assert_close(R_x.float(), torch.tensor(Rx).float())